In [26]:
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from math import sqrt
from scipy import stats
from scipy.stats import norm
from statsmodels.tsa.stattools import adfuller
import pandas_datareader
import math
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
from prettytable import PrettyTable
import cvxpy as cvx
from sklearn.covariance import LedoitWolf
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import requests
import json
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses
from sklearn.svm import SVR
import statsmodels.api as sm
import scipy.optimize as opt
from scipy.cluster.hierarchy import linkage, cut_tree, dendrogram
from scipy.spatial.distance import squareform
from fredapi import Fred
import getFamaFrenchFactors as gff
import quantstats as qs
from sklearn.linear_model import LinearRegression
from tabulate import tabulate
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.base import BaseEstimator

# Describing initial values

In [27]:
#Total years of traing model
years_training=1
# Define the end date
fecha = "2021-01-01"
end=fecha
# Convert the end date string to a datetime object
end_date = datetime.strptime(end, "%Y-%m-%d")
# Calculate the start date as one year before the end date
start_date = end_date - timedelta(days=365*years_training)
# Convert the start date to a string in the same format as the end date
start = start_date.strftime("%Y-%m-%d")

print("Start Date:", start)
print("End Date:", end)

Start Date: 2020-01-02
End Date: 2021-01-01


In [28]:
## define the stocks for our portfolio (via tickers) ###
assets = [ #Healthcare
          "JNJ", #(Johnson & Johnson)
          "PFE", #(Pfizer Inc.)
          "MRK", #(Merck & Co. Inc.)
          "LLY", #(Eli Lilly and company)
          "UNH", #(UnitedHealth Group Incorporated)
          "AMGN", #(Amgen Inc.)
          #Technology
          "MSFT", #(Microsoft Corporation)
          "GOOGL", #(Alphabet Inc. - Google)
          "META", #(Meta Platforms Inc. - Facebook)
          "NVDA", #(Nvidia)
          "AMD", #(Advanced micro devices)
          "TSLA", #(Tesla, Inc.)
          "CRM", #(Salesforce.com Inc.)
          # Finance
          "JPM", #(JPMorgan Chase & Co.)
          "MA", #(Mastercard)
          "BX", #(Blackstone)
          "AXP", #(American Express)
          "GS", #(The Goldman Sachs Group, Inc.)
          "BAC", #(Bank of America Corporation)
          "V", #(Visa Inc.)
          #Consumer goods
          "PG", #(Procter & Gamble Company)
          "KO", #(The Coca-Cola Company)
          "NKE", #(Nike, Inc.)
          "MCD", #(McDonald's Corporation)
          #Energy
          "XOM", #(Exxon Mobil Corporation)
          "CVX", #(Chevron Corporation)
          "SHEL", #(Royal Dutch Shell plc)
          "BP", #(BP p.l.c.)
          "COP", #(ConocoPhillips)
          #Retail
          "AMZN", #(Amazon.com, Inc.)
          "WMT", #(Walmart Inc.)
          "TGT", #(Target Corporation)
          "COST", #(Costco Wholesale Corporation)
          "HD", #(The Home Depot, Inc.)
          #Telecommunications
          "T", #(AT&T Inc.)
          "VZ", #(Verizon Communications Inc.)
          "TMUS", #(T-Mobile US, Inc.)
          #Utilities
          "NEE", #(NextEra Energy, Inc.)
          "DUK", #(Duke Energy Corporation)
          "SO", #(The Southern Company)
          "NRG", #(NRG Energy)
          "EXC", #(Exelon Corporation)
          "AEP", #(American Electric Power Company, Inc.)
          #Real Estate
          "SPG", #(Simon Property Group, Inc.)
          "AMT", #(American Tower Corporation)
          "EQIX", #(Equinix, Inc.)
          "PSA", #(Public Storage)
          "WELL", #(Welltower Inc.)
          #Transportation
          "UPS", #(United Parcel Service, Inc.)
          "FDX", #(FedEx Corporation)
          "DAL", #(Delta Air Lines, Inc.)
          "AAL", #(American Airlines Group Inc.)
          "BLDR", #(Builders first source)
          #"UBER", #(Uber Technologies, Inc.)                  No data before may 2018
          #"ROAD", #(Construction Partners, Inc)               No data before may 2018
          #cripto
          "BTC-USD", #Bitcoin
          "ETH-USD" #Ethereum
          ]

assets =sorted(assets)

## Micro and macroeconomic factors

### FRED

In [29]:
# Initialize Fred with your API key
api_key = '95c774645924b13f93458e248a922d67'
fred = Fred(api_key=api_key)
# Fetch GDP data (quarterly)
gdp = fred.get_series('GDP')

# Fetch Inflation (Consumer Price Index for All Urban Consumers)
inflation = fred.get_series('CPIAUCSL')

# Fetch Unemployment rate (monthly)
unemployment = fred.get_series('UNRATE')

# Fetch Federal Funds Rate (interest rate, monthly)
interest_rates = fred.get_series('FEDFUNDS')

# Combine the data into a DataFrame
macroeconomic_data = pd.concat([gdp, inflation, unemployment, interest_rates], axis=1)
macroeconomic_data.columns = ['GDP', 'Inflation', 'Unemployment', 'Interest Rates']

# Industrial Production Index (a proxy for industry-specific data)
industrial_production = fred.get_series('INDPRO')

# Average Hourly Earnings of All Employees, Total Private
hourly_earnings = fred.get_series('CES0500000003')

# Housing starts (a measure of consumer sentiment in the housing market)
housing_starts = fred.get_series('HOUST')

# Combine with other microeconomic data
microeconomic_data = pd.concat([industrial_production, hourly_earnings, housing_starts], axis=1)
microeconomic_data.columns = ['Industrial Production','Average Hourly Earnings',  'Housing Starts']

In [30]:
# Combine macro and microeconomic data
combined_data = pd.concat([macroeconomic_data, microeconomic_data], axis=1)

# Align the data to a common frequency (e.g., monthly)
combined_data = combined_data.resample('M').ffill()  # Forward-fill to avoid NaN gaps

#Get data only from the start date
combined_data= combined_data[combined_data.index > start]


In [31]:
# Resample to daily frequency
macro_daily = combined_data.resample('D').asfreq()

# Forward-fill NaN values for daily frequency
macro_daily.fillna(method='ffill', inplace=True)

#### Daily data

In [32]:
# U.S. Dollars to Euro Spot Exchange Rate daily
dollar_euro = fred.get_series('DEXUSEU')

# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis
ten_Y = fred.get_series('DEXUSEU')

# Dow Jones Industrial Average
Dow = fred.get_series('DJIA')

# S&P 500
sp500 = fred.get_series('SP500')

# NASDAQ Composite Index
nasdaq = fred.get_series('NASDAQCOM')

# CBOE Gold ETF Volatility Index
gold = fred.get_series('GVZCLS')

# CBOE Crude Oil ETF Volatility Index
oil = fred.get_series('OVXCLS')

# CBOE Russell 2000 Volatility Index
russell = fred.get_series('RVXCLS')

# Combine daily data
daily_data = pd.concat([dollar_euro, ten_Y, Dow,  sp500, nasdaq, gold, oil, russell], axis=1)
daily_data.columns = ['U.S. Dollars to Euro', 'Treasury Securities at 10-Year', 'Dow Jones','S&P 500', 'NASDAQ', 'Gold', 'Crude oil', 'Russell 2000']
#Get data only from the start date
daily_data= daily_data[daily_data.index > start]

In [33]:
# Combine macro and daily data
macro= pd.concat([macro_daily, daily_data], axis=1).dropna()
macro

,GDP,Inflation,Unemployment,Interest Rates,Industrial Production,Average Hourly Earnings,Housing Starts,U.S. Dollars to Euro,Treasury Securities at 10-Year,Dow Jones,S&P 500,NASDAQ,Gold,Crude oil,Russell 2000
2020-01-31,21706.513,258.906,3.6,1.55,101.3768,28.44,1572.0,1.1082,1.1082,28256.03,3225.52,9150.94,13.50,40.35,20.33
2020-02-03,21706.513,258.906,3.6,1.55,101.3768,28.44,1572.0,1.1062,1.1062,28399.81,3248.92,9273.40,12.48,41.81,19.78
2020-02-04,21706.513,258.906,3.6,1.55,101.3768,28.44,1572.0,1.1043,1.1043,28807.63,3297.59,9467.97,12.01,39.44,17.54
2020-02-05,21706.513,258.906,3.6,1.55,101.3768,28.44,1572.0,1.1006,1.1006,29290.85,3334.69,9508.68,11.32,37.88,16.41
2020-02-06,21706.513,258.906,3.6,1.55,101.3768,28.44,1572.0,1.0979,1.0979,29379.77,3345.78,9572.15,11.45,36.44,16.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,28284.498,312.230,3.8,5.33,102.6577,34.68,1321.0,1.0687,1.0687,38460.92,5071.63,15712.75,17.45,29.04,22.34
2024-04-25,28284.498,312.230,3.8,5.33,102.6577,34.68,1321.0,1.0721,1.0721,38085.80,5048.42,15611.76,17.32,28.34,22.16
2024-04-26,28284.498,312.230,3.8,5.33,102.6577,34.68,1321.0,1.0686,1.0686,38239.66,5099.96,15927.90,16.92,27.81,21.04
2024-04-29,28284.498,312.230,3.8,5.33,102.6577,34.68,1321.0,1.0717,1.0717,38386.09,5116.17,15983.08,16.92,27.84,20.63


### EIA api

In [34]:
api_url = 'https://api.eia.gov/v2/steo/data'

params = {"api_key": "whsAp9dwvJg3dXf6Dx1y4BlbFji8hzZExBuZaL4g"}

header = {
    "frequency": "monthly",
    "data": [
        "value"
    ],
    "facets": {
        "seriesId": [
            "COPR_OPEC", #Global crude oil production
            "PAPR_WORLD",
            'BREPUSS',
            'COPR_OPEC',
            'D2TCPUS',
            'PASXPUS',
            'RAIMUUS',
            'T3_STCHANGE_US'
        ]
    },
    "start": start,
    "end": "2024-04",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
    "offset": 0,
    "length": 5000
}

r = requests.get(api_url, params=params, headers={"X-Params": json.dumps(header)})

In [35]:
# Transform data to a dataframe
file = r.json()
eia=pd.DataFrame(file['response']['data'])
# Set the date as a index
eia['period']=pd.to_datetime(eia['period'])
eia.set_index('period', inplace=True)
# Transform the data to have all the macroeconomic data in columns
eai_macro = eia.pivot_table(index='period', columns='seriesId', values='value', aggfunc='first')

In [36]:
# Resample to daily frequency
eai_daily = eai_macro.resample('D').asfreq()

# Forward-fill NaN values for daily frequency
eai_daily.fillna(method='ffill', inplace=True)
eai_daily

seriesId,COPR_OPEC,PAPR_WORLD,PASXPUS,RAIMUUS,T3_STCHANGE_US
period,,,,,
2020-01-01,27.32,101.00411098,1299.893185,53.87,-.58108270968
2020-01-02,27.32,101.00411098,1299.893185,53.87,-.58108270968
2020-01-03,27.32,101.00411098,1299.893185,53.87,-.58108270968
2020-01-04,27.32,101.00411098,1299.893185,53.87,-.58108270968
2020-01-05,27.32,101.00411098,1299.893185,53.87,-.58108270968
...,...,...,...,...,...
2024-03-28,26.895,102.53086846,1212.4401753,78.53,.20367834654
2024-03-29,26.895,102.53086846,1212.4401753,78.53,.20367834654
2024-03-30,26.895,102.53086846,1212.4401753,78.53,.20367834654


In [37]:
assets =sorted(assets)
df_prices = yf.download(assets,start=start,end=end)['Adj Close']
df_prices_all = df_prices.dropna()
market_prices_all = yf.download("^GSPC",start=start,end=end)['Adj Close']
print(start, end)


[*********************100%%**********************]  55 of 55 completed
[*********************100%%**********************]  1 of 1 completed

2020-01-02 2021-01-01


If there is singularity problem like "LinAlgError: SVD did not converge" Check the values of the assets. Could have nan value because there not exist in yahoo

In [38]:
# Join the micro and macroeconomic with all the asset to use it as input in the machine learning
macro_prices = df_prices_all.join(macro, how='inner')  # Join the Fred economic variable with the assets value
df = macro_prices.join(eai_daily, how='inner') # Join the EAI economic variable with the preview table

# Evaluation models

In [39]:
leng_period= 60
#Choose only the assets as output
returns= df[assets]
returns = returns.pct_change(periods=leng_period).shift(-leng_period)
# Split into training and test sets (without using `train_test_split`)
# For training and testing we are using the data leng_periods before because the target will be the mean return of the leng_periods later
train_size = int(0.9 * (len(df)-leng_period))  # Example 80% train, 20% test
#Using all the micro macro and asset as input
X_train = df.iloc[:train_size]
X_test = df.iloc[train_size:(len(df)-leng_period)]
#using only assets return as output
y_train = returns[:train_size]
y_test = returns[train_size:(len(df)-leng_period)]

## Random Forest

In [19]:
# Standardize features for training and test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.transform(df)

# Train a Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate with Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)
prediction = model.predict(X_scaled)
prediction = prediction[-leng_period:].mean(axis=0)

Mean Squared Error: 0.030461813910110276


In [34]:
def random_search_rf(X_train, y_train):
    # Define parameter grid
    param_dist = {
        'n_estimators': randint(100, 1000),
        'max_features': ['auto', 'sqrt'],
        'max_depth': randint(10, 100),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 20),
        'bootstrap': [True, False]
    }

    # Create Random Forest model
    rf = RandomForestRegressor(random_state=42)

    # Define mean squared error as scoring metric
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    # Perform Randomized Search Cross Validation
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                       n_iter=100, scoring=mse_scorer, cv=5, 
                                       random_state=42, n_jobs=-1)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Perform Randomized Search
    random_search.fit(X_train_scaled, y_train)

    # Get best model
    best_rf = random_search.best_estimator_

    return best_rf, random_search.best_params_

best_rf, best_params = random_search_rf(X_train, y_train)
print("Best hyperparameters:", best_params)

Best hyperparameters: {'bootstrap': False, 'max_depth': 26, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 910}


## XGboost

In [35]:
def random_search_xgb(X_train, y_train):
    # Define parameter grid
    param_dist = {
        'max_depth': randint(3, 7),  
        'learning_rate': uniform(0.01, 0.1), 
        'n_estimators': randint(100, 500),  
        'gamma': uniform(0, 0.5), 
        'min_child_weight': randint(1, 5), 
        'subsample': uniform(0.6, 0.3),  
        'colsample_bytree': uniform(0.6, 0.3),  
        'reg_alpha': uniform(0, 1),  
        'reg_lambda': uniform(0, 1) 
    }

    # Create XGBoost model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

    # Define mean squared error as scoring metric
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    # Perform Randomized Search Cross Validation
    random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, 
                                       n_iter=50, scoring=mse_scorer, cv=5, 
                                       random_state=42, n_jobs=-1)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Perform Randomized Search
    random_search.fit(X_train_scaled, y_train)

    # Get best model
    best_xgb = random_search.best_estimator_

    return best_xgb, random_search.best_params_

best_xgb, best_params = random_search_xgb(X_train, y_train)
print("Best hyperparameters:", best_params)

Best hyperparameters: {'colsample_bytree': 0.8659851446851979, 'gamma': 0.013808385936852352, 'learning_rate': 0.06788648955075587, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 441, 'reg_alpha': 0.32815266747473193, 'reg_lambda': 0.1550416167277442, 'subsample': 0.8945522664931593}


## Support Vector Machines (SVM)

In [40]:
def random_search_svm(X_train, y_train):
    # Define parameter grid
    param_dist = {
        'estimator__C': uniform(0.1, 100),
        'estimator__kernel': ['linear', 'rbf', 'poly'],
        'estimator__gamma': ['scale', 'auto']
    }

    # Create SVM model
    svm_model = SVR()

    # Wrap SVM model with MultiOutputRegressor
    multioutput_svm = MultiOutputRegressor(svm_model)

    # Define mean squared error as scoring metric
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    # Perform Randomized Search Cross Validation
    random_search = RandomizedSearchCV(estimator=multioutput_svm, param_distributions=param_dist, 
                                       n_iter=100, scoring=mse_scorer, cv=5, 
                                       random_state=42, n_jobs=-1)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Perform Randomized Search
    random_search.fit(X_train_scaled, y_train)

    # Get best model
    best_svm = random_search.best_estimator_

    return best_svm, random_search.best_params_

best_svm, best_params = random_search_svm(X_train, y_train)
print("Best hyperparameters:", best_params)

Best hyperparameters: {'estimator__C': 9.64101164904113, 'estimator__gamma': 'scale', 'estimator__kernel': 'poly'}


## Neural Networks

In [15]:
def random_search_nn(X_train, y_train):
    # Define parameter grid
    param_dist = {
        'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': np.logspace(-4, 2, 100),
        'learning_rate': ['constant', 'adaptive'],
        'max_iter': randint(100, 1000)
    }

    # Create MLPRegressor model
    nn = MLPRegressor(random_state=42)

    # Define mean squared error as scoring metric
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    # Perform Randomized Search Cross Validation
    random_search = RandomizedSearchCV(estimator=nn, param_distributions=param_dist,
                                       n_iter=100, scoring=mse_scorer, cv=5,
                                       random_state=42, n_jobs=-1)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Perform Randomized Search
    random_search.fit(X_train_scaled, y_train)

    # Get best model
    best_nn = random_search.best_estimator_

    return best_nn, random_search.best_params_

# Example usage:
best_nn, best_params = random_search_nn(X_train, y_train)
print("Best hyperparameters:", best_params)

Best hyperparameters: {'activation': 'tanh', 'alpha': 32.745491628777316, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 289, 'solver': 'adam'}


In [25]:
class KerasRegressorWrapper(BaseEstimator):
    def __init__(self, hidden_layer_sizes=(100,), activation='relu', dropout_rates=(0.0,), learning_rate=0.001, input_dim=None, max_iter=100):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.dropout_rates = dropout_rates
        self.learning_rate = learning_rate
        self.input_dim = input_dim
        self.max_iter = max_iter
        self.model = self.create_model()

    def create_model(self):
        model = Sequential()
        for i, units in enumerate(self.hidden_layer_sizes):
            dropout_rate = self.dropout_rates[min(i, len(self.dropout_rates)-1)]  # Ensure dropout_rates are within bounds
            if i == 0:
                model.add(Dense(units, activation=self.activation, input_dim=self.input_dim))
            else:
                model.add(Dense(units, activation=self.activation))
            model.add(Dropout(dropout_rate))
        model.add(Dense(1))  # Output layer
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        return model

    def fit(self, X, y):
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        self.model.fit(X_scaled, y, epochs=self.max_iter, verbose=0)
        return self

    def predict(self, X):
        scaler = StandardScaler()
        X_scaled = scaler.transform(X)
        return self.model.predict(X_scaled)

def random_search_nn(X_train, y_train):
    # Define parameter grid
    param_dist = {
        'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],
        'activation': ['relu', 'tanh'],
        'dropout_rates': [(0.0,), (0.1, 0.1), (0.2, 0.2, 0.2)],  # Same length as hidden_layer_sizes
        'learning_rate': np.logspace(-4, 2, 100),
        'max_iter': randint(100, 1000)
    }

    # Perform Randomized Search Cross Validation
    random_search = RandomizedSearchCV(estimator=KerasRegressorWrapper(), param_distributions=param_dist,
                                       n_iter=100, scoring='neg_mean_squared_error', cv=5,
                                       random_state=42, n_jobs=-1)

    # Perform Randomized Search
    random_search.fit(X_train, y_train)

    # Get best parameters
    best_params = random_search.best_params_

    # Create and train the best model with the best parameters
    best_model = KerasRegressorWrapper(**best_params)
    best_model.fit(X_train, y_train)

    return best_model, best_params


best_model, best_params = random_search_nn(X_train, y_train)
print("Best hyperparameters:", best_params)

Best hyperparameters: {'activation': 'relu', 'dropout_rates': (0.0,), 'hidden_layer_sizes': (100, 50, 25), 'learning_rate': 2.009233002565046, 'max_iter': 800}
